# PyKX Query Components
Query all components, RDB, HDB, and Gateway.


In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import boto3
import json
import datetime

import pykx as kx

from managed_kx import *
from env_kdb_1 import *

from basictick_setup import *

In [2]:
# triggers credential get
session=None

try:
    # aws: use ada for credentials
    subprocess.call(["which", "ada"])
    os.system(f"ada credentials update --account={ACCOUNT_ID} --provider=isengard --role=Admin --once")
except: 
    None

if AWS_ACCESS_KEY_ID is None:
    print("Using Defaults ...")
    # create AWS session: using access variables
    session = boto3.Session()
else:
    print("Using variables ...")
    session = boto3.Session(
        aws_access_key_id=AWS_ACCESS_KEY_ID,
        aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
        aws_session_token=AWS_SESSION_TOKEN
    )

# create finspace client
client = session.client(service_name='finspace', endpoint_url=ENDPOINT_URL)

Using variables ...


# Current State of HDB

In [3]:
# Query the HDB
hdb = get_pykx_connection(client, 
                          environmentId=ENV_ID, clusterName=HDB_CLUSTER_NAME, 
                          userName=KDB_USERNAME, boto_session=session)

# Dates and Counts
hdb_pdf = hdb("select counts:count i by date from example").pd()

# Number of Rows
hdb_rows = hdb("count example").py()

# Current State of RDB

In [4]:
# Query the RDB
rdb = get_pykx_connection(client, 
                          environmentId=ENV_ID, clusterName=RDB_CLUSTER_NAME, 
                          userName=KDB_USERNAME, boto_session=session)

# Dates and Counts
rdb_head_pdf = rdb("select [5] from example").pd()
rdb_tail_pdf = rdb("select [-5] from example").pd()

# Number of Rows
rdb_rows = rdb("count example").py()

# Current State of GW

In [5]:
# Query the HDB
gw = get_pykx_connection(client, 
                          environmentId=ENV_ID, clusterName=GW_CLUSTER_NAME, 
                          userName=KDB_USERNAME, boto_session=session)

In [6]:
proc_pdf = gw("select process, handle, connected, address from .conn.procs").pd()

# are any processes not connected?
if len(proc_pdf[proc_pdf.connected == False].index) > 0:
    print("reinit Gateway")
    gw("reinit[hdb_name; rdb_name]")
    proc_pdf = gw("select process, handle, connected, address from .conn.procs").pd()

# truncate with elipsis
proc_pdf['address'] = proc_pdf['address'].str.slice(0,110)+"..."

display(proc_pdf)

,process,handle,connected,address
0,rdb,7,True,:ip-192-168-0-211.ec2.internal:5000:GATEWAY_TFC:Host=ip-192-168-0-211.ec2.internal&Port=5000&User=GATEWAY_TFC&...
1,hdb,9,True,:ip-192-168-14-32.ec2.internal:5000:GATEWAY_TFC:Host=ip-192-168-14-32.ec2.internal&Port=5000&User=GATEWAY_TFC&...
2,hdb,10,True,:ip-192-168-3-24.ec2.internal:5000:GATEWAY_TFC:Host=ip-192-168-3-24.ec2.internal&Port=5000&User=GATEWAY_TFC&Ac...


In [7]:
gw("res: `time xasc queryData[`example;`]").pd()

gw_pdf = gw("res").pd()
gw_head_pdf = gw("select [5] from res").pd()
gw_tail_pdf = gw("select [-5] from res").pd()

# Number of Rows
gw_rows = len(gw_pdf.index)

# HDB Contents

In [8]:
display(hdb_pdf)

# Number of Rows
print(f"Rows: {hdb_rows:,}")

,counts
date,
2023-04-14,1000000
2023-04-15,1000000
2023-04-16,1000000
2023-04-17,1000000
2023-04-18,1000000
2023-04-19,1000000
2023-04-20,1000000
2023-04-21,1000000
2023-04-22,1000000


Rows: 10,000,000


# RDB Contents

In [9]:
display(rdb_head_pdf)
display(rdb_tail_pdf)

# Number of Rows
print(f"Rows: {rdb_rows:,}")

,sym,time,number
0,ilm,2023-06-19 19:03:45.873035288,73
1,kjl,2023-06-19 19:03:45.873035288,76
2,igi,2023-06-19 19:03:45.873035288,21
3,nao,2023-06-19 19:03:45.873035288,41
4,jde,2023-06-19 19:03:45.883033520,42


,sym,time,number
0,ile,2023-06-19 19:39:49.853034839,36
1,igd,2023-06-19 19:39:49.863039873,25
2,ach,2023-06-19 19:39:49.863039873,71
3,obi,2023-06-19 19:39:49.863039873,12
4,cog,2023-06-19 19:39:49.863039873,96


Rows: 973,044


# GW Contents

In [10]:
display(gw_head_pdf)
display(gw_tail_pdf)

# Number of Rows
print(f"Rows: {gw_rows:,}")

# Note: RDB will have less than GW  b/c data is always arriving to the RDB
# RDB was queried first and then again as part of GW query

,sym,time,number
0,aaa,2023-04-14 23:17:28.473,53231
1,aaa,2023-04-14 23:17:28.473,153560
2,aaa,2023-04-14 23:17:28.473,449428
3,aaa,2023-04-14 23:17:28.473,631966
4,aaa,2023-04-14 23:17:28.473,941566


,sym,time,number
0,mfc,2023-06-19 19:39:50.863037952,62
1,cgb,2023-06-19 19:39:50.863037952,64
2,adn,2023-06-19 19:39:50.863037952,51
3,meb,2023-06-19 19:39:50.863037952,62
4,ccc,2023-06-19 19:39:50.863037952,66


Rows: 10,973,485


In [11]:
print( f"Last Run: {datetime.datetime.now()}" )

Last Run: 2023-06-19 19:39:54.682272
